In [43]:
from dataclasses import dataclass
from pathlib import Path
import torch
import torch.nn as nn
from jaxtyping import Float, Int

In [44]:
def get_gutenberg_book(
	id: int|None = 84,
	data_temp: Path|str = "../../../../data/gutenberg_data",
	remove_gutenberg_meta: bool = True,
) -> str:
	
	data_temp = Path(data_temp)
	data_temp.mkdir(parents=True, exist_ok=True)
	
	url: str = f"https://www.gutenberg.org/cache/epub/{id}/pg{id}.txt"
	data_path: Path = Path(data_temp) / f"{id}.txt"
	data: str
	# read from cache if it exists
	if data_path.exists():
		with open(data_path, 'r', encoding='utf-8') as file:
			data = file.read()
	else:
		# download if it doesn't exist
		response = requests.get(url)
		response.raise_for_status()  # Ensure that the download was successful
		data = response.text

		# save to cache
		with open(data_path, 'w', encoding='utf-8') as file:
			file.write(data)

	# remove header/footer
	if remove_gutenberg_meta:
		data = '***'.join(data.split('***')[2:])
		data = '***'.join(data.split('***')[:-1])
	
	return data

def get_many_books(
		ids: list[int],
		data_temp: Path|str = "../data/gutenberg_data",
	) -> list[str]:
	
	data: list[str] = []
	for id in ids:
		print(f"Getting book {id}...")
		item: str = get_gutenberg_book(id, data_temp)
		print(f"\t{len(item)} characters read")
		data.append(item)
	
	return data

In [45]:
@dataclass
class Config():
    d_model: int
    d_vocab: int
    d_hidden: int

In [46]:
class MLP(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.linear1 = nn.Linear(config.d_model, config.d_hidden)
        self.linear2 = nn.Linear(config.d_hidden, config.d_model)
        

    def forward(self, x):
        x = self.linear1(x)
        x = torch.relu(x)
        x = self.linear2(x)
        return x

In [47]:
class AttentionHead(nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.config = config
        self.W_qk = nn.Linear(config.d_model, config.d_model)
        self.W_vo = nn.Linear(config.d_model, config.d_model)
        self.softmax = nn.Softmax(dim=-1)
        

    def create_mask(self, n_c: int) -> torch.Tensor:
        mask = torch.triu(-1 * torch.inf * torch.ones(n_c, n_c), diagonal=0)
        return mask

    def forward(self, x):
        #create mask, with size n_c x n_c
        mask = self.create_mask(x.shape[1])

        #compute attention scores
        A = self.softmax((self.W_qk(x)) @ x.transpose(-1, -2) + mask) @ self.W_vo(x)
        return A

In [48]:
# Attention head test
x = torch.ones(1, 5, 16)
config = Config(d_model=16, d_vocab=1000, d_hidden=64, d_head=8)
attention_head = AttentionHead(config)
output = attention_head.forward(x)
print(output.shape)

TypeError: Config.__init__() got an unexpected keyword argument 'd_head'

In [ ]:
class TransformerBlock(torch.nn.Module):
    def __init__(self, config: Config):
        super().__init__()
        self.attention_head = AttentionHead(config)
        self.mlp = MLP(config)

    def forward(self, x):
        return x + self.attention_head(x) + self.mlp(x)

In [52]:
class Transformer(torch.nn.Module):
    def __init__(self, num_blocks: int, config: Config):
        super().__init__()
        self.config = config
        self.embedding = nn.Linear(config.d_vocab, config.d_model)
        self.blocks = nn.ModuleList([TransformerBlock(config) for _ in range(num_blocks)])
        

    def forward(self, x):
        x = self.embedding(x)
        for block in self.blocks:
            x = block(x)
        x = self.embedding(x).transpose(1, 2)
        return x
    

In [55]:
# Test the whole thing
config = Config(d_model=16, d_vocab=1000, d_hidden=64)
transformer = Transformer(num_blocks=2, config=config)
x = torch.ones(1, 5, config.d_vocab, dtype=torch.float)
y = transformer(x)
print(y.shape)
print(y)
print(x)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (5x16 and 1000x16)